# Reactive Flash Validation: Based on Marano & Holder (1997)

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from stages.reactive_stage import reactive_stage_rule
from stages.non_reactive_stage import non_reactive_stage_rule

model = pe.ConcreteModel()

# Validate 3 cases of Mobil F-T reactor effluent

In [3]:
model.TRAY = pe.RangeSet(1,3)

### Construct a single reactive flash stage

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

In [5]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


### Fixing In/Product Redundent Flow

In [6]:
for j in model.reactive:   
    model.reactive[j].x_.fix(0)
    model.reactive[j].y_.fix(0)
    model.reactive[j].L['in'].fix(0)
    model.reactive[j].L['P'].fix(0)
    model.reactive[j].V['in'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].H_L_.fix(0)
    model.reactive[j].H_V_.fix(0)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].F.fix(1)
    model.reactive[j].z['CO'].fix(1/(1+0.679)-0.001/2)
    model.reactive[j].z['H2'].fix(0.679/(1+0.679)-0.001/2)
    model.reactive[j].z['C30H62'].fix(0.001)

### Input Model Parameters

In [7]:
model.reactive[1].cat.fix(580)
model.reactive[1].P.fix(14.8)
model.reactive[1].T.setub(260+273.15)
model.reactive[1].VLE_block.n_ave.fix(27.8)

model.reactive[2].cat.fix(150)
model.reactive[2].P.fix(25.15)
model.reactive[2].T.setub(256+273.15)
model.reactive[2].VLE_block.n_ave.fix(41)

model.reactive[3].cat.fix(440)
model.reactive[3].P.fix(14.73)
model.reactive[3].T.setub(244+273.15)
model.reactive[3].VLE_block.n_ave.fix(58.4)

In [8]:
check_DOF(pe,model)

Active Equality Constraints:	 3543
Active Inequality Constraints:	 0
Active Variables:		 4062
Fixed Variables:		 516
DOF:				 3


### Solve

In [9]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [10]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma97'
# opt.options['nlp_scaling_method'] = None
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma97
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma97                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma97.

Number of nonzeros in equality constraint Jacobian...:    11555
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

 246r-1.4194500e+03 1.69e+01 9.99e+02   1.2 0.00e+00    -  0.00e+00 0.00e+00R  1
 247r-1.4194500e+03 1.69e+01 9.99e+02   1.2 1.08e+05    -  3.59e-07 3.79e-07f  1
 248r-1.4194501e+03 1.69e+01 1.64e+03   1.2 5.70e+04    -  1.82e-06 3.82e-06f  1
 249r-1.4194504e+03 1.69e+01 2.59e+04   1.2 2.11e+04    -  5.40e-05 1.08e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250r-1.4194539e+03 1.69e+01 3.19e+04   1.2 5.14e+03    -  1.63e-04 1.23e-04f  1
 251r-1.4194682e+03 1.69e+01 3.87e+04   1.2 1.89e+03    -  6.57e-04 5.12e-04f  1
 252r-1.4195202e+03 1.69e+01 3.98e+04   1.2 1.75e+03    -  4.94e-03 1.87e-03f  1
 253r-1.4196583e+03 1.69e+01 1.31e+05   1.2 1.70e+03    -  1.76e-02 5.10e-03f  1
 254r-1.4199547e+03 1.69e+01 2.26e+05   1.2 1.44e+03    -  4.85e-02 1.27e-02f  1
 255r-1.4211229e+03 1.69e+01 4.73e+04   1.2 1.13e+03    -  7.42e-02 6.24e-02f  1
 256r-1.4229593e+03 1.69e+01 1.57e+05   1.2 9.16e+02    -  6.74e-02 1.21e-01f  1
 257r-1.4243421e+03 1.69e+01

 349r-1.4218241e+03 1.11e+00 1.00e+03   0.0 0.00e+00    -  0.00e+00 0.00e+00R  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 350r-1.4218081e+03 1.08e+00 8.07e+05   0.0 5.28e+02    -  1.22e-01 3.20e-03f  1
 351r-1.4212481e+03 4.02e-02 5.62e+06   0.0 5.22e+02    -  5.70e-01 1.16e-01f  1
 352 -1.4212650e+03 3.95e-02 3.76e+03  -1.0 1.75e+02    -  1.21e-02 1.66e-02h  1
 353 -1.4212586e+03 3.95e-02 6.51e+04  -1.0 9.38e+02    -  6.38e-02 9.42e-04h  1
 354r-1.4212586e+03 3.95e-02 1.00e+03  -1.0 0.00e+00    -  0.00e+00 3.00e-07R  6
 355r-1.4212587e+03 3.60e-02 3.31e+05  -1.0 1.92e+02    -  7.26e-02 2.75e-04f  1
 356r-1.4212918e+03 2.40e-01 2.84e+06  -1.0 1.11e+02    -  2.78e-01 3.66e-02f  1
 357r-1.4214041e+03 6.02e-01 4.91e+06  -1.0 4.87e+01    -  4.85e-01 1.39e-01f  1
 358r-1.4219377e+03 6.46e-01 3.76e+06  -1.0 6.10e+01    -  5.12e-01 4.38e-01f  1
 359r-1.4229673e+03 4.08e-01 2.91e+06  -1.0 8.53e+01    -  3.09e-01 5.15e-01f  1
iter    objective    inf_pr 

 441 -1.4259939e+03 2.98e-03 7.66e+10  -1.0 5.53e+02    -  1.00e+00 1.48e-03h  9
 442 -1.4259766e+03 3.00e-03 8.04e+10  -1.0 6.45e+02    -  1.51e-01 1.52e-03h  9
 443 -1.4214930e+03 1.56e+00 2.32e+13  -1.0 6.57e+02    -  1.00e+00 3.92e-01w  1
 444 -1.4222833e+03 4.11e+00 1.28e+14  -1.0 9.44e+01    -  7.88e-02 9.36e-01w  1
 445 -1.4223516e+03 5.50e-02 1.49e+13  -1.0 1.22e+01    -  1.24e-01 9.91e-01w  1
 446 -1.4259591e+03 3.02e-03 8.04e+10  -1.0 1.63e+01    -  1.00e+00 1.53e-03h  8
 447 -1.4259413e+03 3.03e-03 8.42e+10  -1.0 7.22e+02    -  1.55e-01 1.55e-03h  9
 448 -1.4259237e+03 3.05e-03 8.42e+10  -1.0 7.28e+02    -  1.00e+00 1.53e-03h  9
 449 -1.4259065e+03 3.07e-03 8.42e+10  -1.0 7.65e+02    -  1.59e-01 1.45e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 450 -1.4258894e+03 3.08e-03 8.79e+10  -1.0 7.70e+02    -  1.00e+00 1.44e-03h  9
 451 -1.4258723e+03 3.10e-03 8.79e+10  -1.0 7.97e+02    -  1.63e-01 1.38e-03h  9
 452 -1.4258551e+03 3.11e-03

 538 -1.5793640e+03 1.85e-02 3.17e+05  -5.7 2.36e+01    -  6.92e-01 2.70e-01h  1
 539 -1.5794500e+03 1.08e-02 3.70e+06  -5.7 2.46e+01    -  7.50e-02 5.92e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540 -1.5794500e+03 1.07e-02 3.69e+06  -5.7 1.70e-02    -  7.59e-04 4.31e-03h  1
 541 -1.5794500e+03 1.00e-02 3.46e+06  -5.7 1.33e-02    -  9.99e-01 6.25e-02f  5
 542 -1.5794500e+03 3.40e-07 7.84e+04  -5.7 1.25e-02    -  6.76e-02 1.00e+00h  1
 543 -1.5794500e+03 1.43e-10 1.07e+00  -5.7 4.36e-04    -  1.00e+00 1.00e+00h  1
 544 -1.5794500e+03 1.45e-10 5.55e-07  -5.7 1.91e-04    -  1.00e+00 1.00e+00h  1
 545 -1.5794500e+03 1.44e-10 1.08e-03  -8.6 3.12e-04    -  1.00e+00 1.00e+00h  1
 546 -1.5794500e+03 2.91e-10 2.80e-14  -8.6 1.31e-08    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 546

                                   (scaled)                 (unscaled)
Objective...............:  -1.5794500157869822e+03   -1.5794500157869822e+03
Dual infea

# Calculate Reactor Effluent (CT-256-3)

In [11]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.reactive[1].T.value-273.15))
print('Conversion(H2+CO): \t{:.2f}\t\t'.format(1-((model.reactive[1].y['CO'].value+model.reactive[1].y['H2'].value)*model.reactive[1].V['out'].value)))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.reactive[1].L['out'].value+model.reactive[1].V['out'].value))
print('-'*72)
for i in model.reactive[1].x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.reactive[1].x[i].value*model.reactive[1].L['out'].value\
        +model.reactive[1].y[i].value*model.reactive[1].V['out'].value)/(model.reactive[1].L['out'].value+model.reactive[1].V['out'].value)))

Temperature (C): 	260.00		
Conversion(H2+CO): 	0.85		
Total FLow (kmol/s): 	0.50		
------------------------------------------------------------------------
H2         		13.3157%
CO         		16.1067%
CO2        		48.8147%
H2O        		4.5129%
C2H4       		2.4120%
C3H6       		2.0183%
C4H8       		1.3047%
C5H10      		0.7351%
C6H12      		0.5000%
C7H14      		0.3402%
C8H16      		0.2314%
C9H18      		0.1574%
C10H20     		0.1071%
C11H22     		0.0464%
C12H24     		0.0316%
C13H26     		0.0215%
C14H28     		0.0146%
C15H30     		0.0099%
C16H32     		0.0068%
C17H34     		0.0046%
C18H36     		0.0031%
C19H38     		0.0021%
C20H40     		0.0014%
C1H4       		5.4520%
C2H6       		1.2968%
C3H8       		0.5046%
C4H10      		0.4116%
C5H12      		0.4324%
C6H14      		0.2941%
C7H16      		0.2001%
C8H18      		0.1361%
C9H20      		0.0926%
C10H22     		0.0630%
C11H24     		0.0693%
C12H26     		0.0471%
C13H28     		0.0321%
C14H30     		0.0218%
C15H32     		0.0148%
C16H34     		0.0101%
C17H36     		0.0069%
C

# Calculate Reactor Effluent (CT-256-4)

In [12]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.reactive[2].T.value-273.15))
print('Conversion(H2+CO): \t{:.2f}\t\t'.format(1-((model.reactive[2].y['CO'].value+model.reactive[2].y['H2'].value)*model.reactive[2].V['out'].value)))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.reactive[2].L['out'].value+model.reactive[2].V['out'].value))
print('-'*72)
for i in model.reactive[2].x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.reactive[2].x[i].value*model.reactive[2].L['out'].value\
        +model.reactive[2].y[i].value*model.reactive[2].V['out'].value)/(model.reactive[2].L['out'].value+model.reactive[2].V['out'].value)))

Temperature (C): 	256.00		
Conversion(H2+CO): 	0.41		
Total FLow (kmol/s): 	0.76		
------------------------------------------------------------------------
H2         		32.3890%
CO         		45.1366%
CO2        		16.1901%
H2O        		1.0296%
C2H4       		0.6963%
C3H6       		0.6017%
C4H8       		0.4017%
C5H10      		0.2337%
C6H12      		0.1642%
C7H14      		0.1153%
C8H16      		0.0810%
C9H18      		0.0569%
C10H20     		0.0400%
C11H22     		0.0179%
C12H24     		0.0126%
C13H26     		0.0088%
C14H28     		0.0062%
C15H30     		0.0044%
C16H32     		0.0031%
C17H34     		0.0022%
C18H36     		0.0015%
C19H38     		0.0011%
C20H40     		0.0007%
C1H4       		1.5239%
C2H6       		0.3743%
C3H8       		0.1504%
C4H10      		0.1267%
C5H12      		0.1375%
C6H14      		0.0966%
C7H16      		0.0678%
C8H18      		0.0477%
C9H20      		0.0335%
C10H22     		0.0235%
C11H24     		0.0267%
C12H26     		0.0188%
C13H28     		0.0132%
C14H30     		0.0093%
C15H32     		0.0065%
C16H34     		0.0046%
C17H36     		0.0032%
C

# Calculate Reactor Effluent (CT-256-5)

In [13]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.reactive[3].T.value-273.15))
print('Conversion(H2+CO): \t{:.2f}\t\t'.format(1-((model.reactive[3].y['CO'].value+model.reactive[3].y['H2'].value)*model.reactive[3].V['out'].value)))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.reactive[3].L['out'].value+model.reactive[3].V['out'].value))
print('-'*72)
for i in model.reactive[3].x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.reactive[3].x[i].value*model.reactive[3].L['out'].value\
        +model.reactive[3].y[i].value*model.reactive[3].V['out'].value)/(model.reactive[3].L['out'].value+model.reactive[3].V['out'].value)))

Temperature (C): 	244.00		
Conversion(H2+CO): 	0.57		
Total FLow (kmol/s): 	0.66		
------------------------------------------------------------------------
H2         		28.4629%
CO         		37.3009%
CO2        		25.5309%
H2O        		2.1197%
C2H4       		0.7470%
C3H6       		0.7051%
C4H8       		0.5141%
C5H10      		0.3267%
C6H12      		0.2507%
C7H14      		0.1924%
C8H16      		0.1476%
C9H18      		0.1133%
C10H20     		0.0869%
C11H22     		0.0425%
C12H24     		0.0326%
C13H26     		0.0250%
C14H28     		0.0192%
C15H30     		0.0147%
C16H32     		0.0113%
C17H34     		0.0087%
C18H36     		0.0067%
C19H38     		0.0051%
C20H40     		0.0039%
C1H4       		1.4969%
C2H6       		0.4016%
C3H8       		0.1763%
C4H10      		0.1622%
C5H12      		0.1922%
C6H14      		0.1475%
C7H16      		0.1132%
C8H18      		0.0868%
C9H20      		0.0666%
C10H22     		0.0511%
C11H24     		0.0634%
C12H26     		0.0487%
C13H28     		0.0373%
C14H30     		0.0287%
C15H32     		0.0220%
C16H34     		0.0169%
C17H36     		0.0129%
C

# Vapor Liquid Separation (CT-256-4)

In [14]:
print('Component\t\t Liquid:\t\t\tVapor:\t\t\tlog K')
print('-'*108)
for i in model.reactive[2].x:
    print('{:7s}\t\t\t{:8.4%}\t\t\t{:8.4%}\t\t{:.4f}'.format(i,model.reactive[2].x[i].value,model.reactive[2].y[i].value,np.log10(model.reactive[2].y[i].value/model.reactive[2].x[i].value)))

Component		 Liquid:			Vapor:			log K
------------------------------------------------------------------------------------------------------------
H2     			 2.5602%			32.4351%		1.1027
CO     			 4.0836%			45.2001%		1.0441
CO2    			 3.0869%			16.2104%		0.7203
H2O    			 0.8905%			 1.0298%		0.0631
C2H4   			 0.1760%			 0.6971%		0.5978
C3H6   			 0.2878%			 0.6022%		0.3206
C4H8   			 0.1733%			 0.4020%		0.3655
C5H10  			 0.1971%			 0.2338%		0.0741
C6H12  			 0.2454%			 0.1641%		-0.1748
C7H14  			 0.2869%			 0.1151%		-0.3968
C8H16  			 0.3212%			 0.0807%		-0.6002
C9H18  			 0.3482%			 0.0565%		-0.7901
C10H20 			 0.3682%			 0.0395%		-0.9697
C11H22 			 0.2429%			 0.0176%		-1.1412
C12H24 			 0.2472%			 0.0122%		-1.3062
C13H26 			 0.2475%			 0.0085%		-1.4660
C14H28 			 0.2441%			 0.0058%		-1.6214
C15H30 			 0.2372%			 0.0040%		-1.7731
C16H32 			 0.2268%			 0.0027%		-1.9216
C17H34 			 0.2131%			 0.0018%		-2.0674
C18H36 			 0.1966%			 0.0012%		-2.2108
C19H38 			 0.1775%			 0.0008%		-2.3522
C20H